# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.1-70b-versatile"

## Creating Crew, Agents, and Tasks

In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: To create a comprehensive report on the latest news and updates related to inflation in the US and its impact on the stock market in 2024, I need to gather information from reliable financial news sources and real-time market data. My first step will be to search the internet for recent news articles and updates on US inflation and its effects on the stoc



# Agent: Lead Market Analyst
## Thought: Thought: I have now gathered more information from the Reuters article about the recent US inflation news and its impact on the stock market. The article highlights that US consumer prices increased by the most in nine months in December, driven by higher costs for energy goods. However, there are hopeful signs in the fight against inflation, with underlying price pressures subsiding. The Federal Reserve is expected to hold interest rates steady at its upcoming meeting, but financial markets are betting on a rate cut in June.
To further analyze the impact of inflation on the stock market, I should read more articles from other news sources.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.usbank.com/investing/financial-perspectives/market-news/federal-reserve-tapering-asset-purchases.html\"}"
## Tool Output: 
Federal Reserve Recalibrates Monetary Policy as Inflation Recedes | U.S. Bank
Skip to main content
L



# Agent: Lead Market Analyst
## Thought: Thought: I have now gathered more information from the U.S. Bank article about the Federal Reserve's monetary policy and its impact on the economy. The article highlights that the Federal Reserve cut interest rates by 0.25% at its December 2024 meeting, resulting in total rate cuts of 1.00% since September. The FOMC now projects just two interest rate cuts in 2025, compared to earlier projections of four rate cuts. 
To further analyze the impact of inflation on the stock market, I should read more articles from other news sources.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://finance.yahoo.com/news/inflation-and-retail-sales-data-greet-a-roaring-stock-market-rally-what-to-know-this-week-124552744.html\"}"
## Tool Output: 
Inflation and retail sales data greet a roaring stock market rally: What to know this week
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phon



# Agent: Lead Market Analyst
## Final Answer: 
The current state of the US economy is characterized by rising consumer prices, driven by higher energy costs, and a Federal Reserve that is recalibrating its monetary policy in response to receding inflation. The Fed cut interest rates by 0.25% in December 2024, resulting in total rate cuts of 1.00% since September, and now projects just two interest rate cuts in 2025. The stock market has reacted positively to the inflation data and the Fed's policy decisions, with the S&P 500 and the Dow Jones Industrial Average rising more than 4.5% in the week following the presidential election. 

The key points to watch in the coming week include the release of the October Consumer Price Index (CPI) and retail sales data. The CPI is expected to rise 2.6% annually in October, an increase from the 2.4% rise in September, while retail sales are expected to increase 0.3% over the prior month during October. 

The Federal Reserve's next meeting is sche



# Agent: Chief Data Strategist
## Thought: Thought: To create a comprehensive analysis report on the impact of inflation in the US on the stock market in 2024, I need to gather accurate and up-to-date information from reliable sources. My first step is to search the internet for relevant articles, reports, and data on this topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"inflation in US and stock market 2024\"}"
## Tool Output: 

Search results: Title: Core inflation rate slows to 3.2% in December, less than expected
Link: https://www.cnbc.com/2025/01/15/cpi-inflation-december-2024-.html
Snippet: The consumer price index was expected to rise 0.3% in December and show an annual inflation rate of 2.9%.
---
Title: Analysis: Assessing Inflation's Impact - U.S. Bank
Link: https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html
Snippet: For the 12-month period ending in November 2024, the Consumer 



# Agent: Chief Data Strategist
## Thought: Thought: I have gathered a significant amount of information on the impact of inflation on the stock market in 2024, including data on stock market performance and investor sentiment. However, to create a comprehensive analysis report, I need to ensure that I have considered all relevant factors and perspectives.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US economy inflation stock market 2024 analysis\"}"
## Tool Output: 

Search results: Title: US economy ends 2024 on a high as output growth accelerates
Link: https://www.spglobal.com/marketintelligence/en/mi/research-analysis/us-economy-ends-2024-on-a-high-as-output-growth-accelerates-Jan25.html
Snippet: 2024 ends on a high​​ The solid PMI reading follows news that the US economy grew at a stronger than previously though rate in the third quarter, ...
---
Title: Analysis: How will inflation, interest rates and the U.S. consumer fare ...
Link: https://www.pbs.o



# Agent: Creative Content Director
## Thought: Thought: I've gathered a variety of current insights on inflation in the US and its impact on the stock market in 2024. This information can be used to create engaging content that educates the target audience.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"US inflation impact on stock market 2024 analysis\", \"website\": \"https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html\"}"
## Tool Output: 
Relevant Content:
2021, 6.5% in 2022 and 3.4% in 2023. Through November 2024, inflation over the previous 12 months was 2.7%.6 Still, headline inflation remains above the Federal Reserve’s 2% target.4 Tags: Market commentary Share: Copy link to clipboard Related articles Is a market correction coming? Having reached new record highs more than 50 times in 2024, investors wonder if the S&P 500 is due for a pullback in 2025. Continue reading Economy 



# Agent: Chief Content Officer
## Final Answer: 
```markdown
# Comprehensive Analysis Report: Inflation in the US and its Impact on the Stock Market in 2024

## Introduction  
As of late 2024, inflation in the US has significantly cooled from its recent highs but remains a critical topic impacting investor behavior and market dynamics. The latest Consumer Price Index (CPI) data indicates that inflation has stabilized at **2.7%** for the year ending November 2024, a notable decline from the **9.1%** peak observed in mid-2022. This report delves into the implications of ongoing inflation on the stock market, as well as investor sentiment toward various asset classes.

Understanding the complexities of inflation is vital for investors navigating the stock market. Despite a reduction in headline inflation, core inflation—excluding volatile food and energy prices—remains steady at **3.3%**. This persistence of price pressures in essential sectors such as housing and transportation could s

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
📊 Understanding inflation's impact on the
#StockMarket is crucial for investors! Inflation
stabilizes at 2.7% in Nov 2024. Learn more in our
latest report! #Investing #Finance
--------------------------------------------------
LinkedIn
As inflation cools in the US, with a steady rate
of 2.7%, what does this mean for the stock market?
Our comprehensive report offers insights and
analysis for savvy investors. #Investing
#StockMarket #Inflation
--------------------------------------------------
Instagram
💡 Inflation vs. Stock Market! Did you know that
core inflation is at 3.3%? This might affect your
investments. Stay informed! 📈💰 #Finance
#InvestSmart #EconomicTrends
--------------------------------------------------


## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Comprehensive Analysis Report: Inflation in the US and its Impact on the Stock Market in 2024

## Introduction  
As of late 2024, inflation in the US has significantly cooled from its recent highs but remains a critical topic impacting investor behavior and market dynamics. The latest Consumer Price Index (CPI) data indicates that inflation has stabilized at **2.7%** for the year ending November 2024, a notable decline from the **9.1%** peak observed in mid-2022. This report delves into the implications of ongoing inflation on the stock market, as well as investor sentiment toward various asset classes.

Understanding the complexities of inflation is vital for investors navigating the stock market. Despite a reduction in headline inflation, core inflation—excluding volatile food and energy prices—remains steady at **3.3%**. This persistence of price pressures in essential sectors such as housing and transportation could signal underlying economic challenges that may influence stock market performance.

---

## Key Findings

### Inflation Trends  
- Recent reports reveal that while headline inflation has decreased, **core inflation** remains a point of concern. The **3.3%** figure suggests that essential goods and services are still experiencing price increases, which could affect consumer spending patterns.

- This distinction between headline and core inflation is crucial for investors. As core inflation influences the buying power of consumers, sectors dependent on discretionary spending could be at risk if prices continue to rise in essential categories.

### Monetary Policy Response  
- The Federal Reserve has proactively responded to inflationary pressures by enacting multiple interest rate cuts to stimulate economic activity. Since September 2024, the Fed has cut rates by a cumulative **1 percentage point**, a move that could pave the way for further adjustments in **2025**.

- Markets are closely monitoring these developments, assessing the effectiveness of the Fed's measures in curtailing inflation. An expected trend in monetary easing could provide further support to the stock market, encouraging investment and risk-taking behavior among investors.

### Stock Market Performance  
- In 2024, the **S&P 500** achieved unprecedented highs, driven partly by investor optimism stemming from the Fed's recent easing policies. The index surged by more than **4.5%** following the presidential election, reflecting heightened market confidence in economic stability.

- Moreover, the correlation between monetary policy changes and stock market movements emphasizes the importance of closely following Fed announcements. Investors are likely to adjust their strategies based on anticipated economic indicators and Fed communications—stay informed for timely adjustments to investment approaches.

---

## Data Visualizations

### Inflation Rate Chart (2022-2024)  
![Inflation Rate Chart](link-to-chart)  
A graphical representation shows the decline of inflation from peak levels in 2022 to the current stabilization at **2.7%**.

### Stock Market Performance Graph  
![Stock Market Performance Graph](link-to-graph)  
This graph outlines the S&P 500's performance in 2024 alongside key inflation data points, illustrating the correlation between monetary policy changes and stock market movements.

### Comparative Table of Forecasted Inflation Rates  
| Institution         | Projected Inflation Rate |
|---------------------|--------------------------|
| U.S. Bank           | 2.5%                     |
| Vanguard            | 2.6%                     |
| J.P. Morgan         | 2.8%                     |

---

## Multimedia Content Creation Suggestions

1. **Blog Series:** Develop a series of articles dissecting the influence of inflation on different sectors, including technology, healthcare, and consumer goods. Each article should focus on detailed analysis regarding which sectors are likely to perform well under prevailing conditions.

2. **Social Media Campaign:** Launch an engaging campaign utilizing infographics and educational posts to explain the relationship between inflation rates, Federal Reserve actions, and stock market performance. This initiative aims to elevate financial literacy among followers and enhance audience engagement on platforms such as Twitter, Instagram, and LinkedIn.

3. **Video Interviews:** Create a video series featuring market analysts discussing trends in inflation and their implications. Providing practical investment advice through these discussions can empower viewers and foster a deeper understanding of current economic conditions.

---

## Conclusion  
Inflation continues to be a driving force behind economic conditions in the United States, influencing both consumer behavior and stock market dynamics. As the Federal Reserve navigates its monetary policy, investors must remain vigilant, utilizing informed strategies to capitalize on opportunities while mitigating risks associated with fluctuating inflation rates.

To navigate this complex landscape effectively, staying updated with ongoing economic developments and consulting with financial professionals will be essential for making educated investment choices. This report aims to provide a thorough understanding of the current economic environment, allowing stakeholders to better navigate the intersections of inflation and market performance as we move into **2025**.

---